In [1]:
import sys
sys.path.append('E:/xinlib')
import numpy as np
from xintab import DataLoader
import xintab

import mxnet as mx
from mxnet import gluon, autograd, nd
from mxnet.gluon import nn

batch_size = 256

Using CNTK backend
C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\cntk_backend.py:21: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


In [26]:
mnist.dense0.name_scope?

In [39]:
class Xin(nn.Block):
    '''
    nn.Block主要提供：
        存储参数
        描述 forward 如何执行
        自动求导
    '''
    def __init__(self, **kwargs):
        '''
        prefix：表示给里面的所有层和参数的名字加上前缀（prefix）使得他们在系统里面独一无二；
        例如：mnist = Xin(prefix='mnist_') -> mnist.dense0.name 是 'mnist_dense0'
        '''
        super().__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)

    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))

In [40]:
class Sequential(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def add(self, block):
        self._children.append(block)
    def forward(self, x):
        for block in self._children:
            x = block(x)
        return x

In [47]:
class RecMLP(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(nn.Dense(256, activation="relu"))
            self.net.add(nn.Dense(128, activation="relu"))
            self.dense = nn.Dense(64)

    def forward(self, x):
        return nd.relu(self.dense(self.net(x)))

rec_mlp = nn.Sequential(prefix='A')
rec_mlp.add(RecMLP(prefix='B'))
rec_mlp.add(nn.Dense(10))
print(rec_mlp)

Sequential(
  (0): RecMLP(
    (net): Sequential(
      (0): Dense(None -> 256, Activation(relu))
      (1): Dense(None -> 128, Activation(relu))
    )
    (dense): Dense(None -> 64, linear)
  )
  (1): Dense(None -> 10, linear)
)


In [64]:
rec_mlp[0]

RecMLP(
  (net): Sequential(
    (0): Dense(None -> 256, Activation(relu))
    (1): Dense(None -> 128, Activation(relu))
  )
  (dense): Dense(None -> 64, linear)
)

In [67]:
a = gluon.Parameter('a' ,shape=(3, 3))
a.initialize()

In [70]:
a.data()


[[ 0.04302466 -0.01240446  0.0096741 ]
 [-0.05040384 -0.01299434 -0.06534886]
 [-0.06031662  0.06756049  0.02764003]]
<NDArray 3x3 @cpu(0)>

In [73]:
a.grad()


[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
<NDArray 3x3 @cpu(0)>

In [75]:
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def forward(self, x):
        return x - x.mean()

In [76]:
net = CenteredLayer()
x = nd.array([3, 4, 5])
net(x)


[-1.  0.  1.]
<NDArray 3 @cpu(0)>

In [83]:
net1 = nn.Sequential()
with net1.name_scope():
    net1.add(nn.Dense(128, activation='relu'))
    net1.add(nn.Dense(10, activation='relu'))
    net1.add(CenteredLayer())

In [84]:
net1

Sequential(
  (0): Dense(None -> 128, Activation(relu))
  (1): Dense(None -> 10, Activation(relu))
  (2): CenteredLayer(
  
  )
)

In [85]:
pd = gluon.ParameterDict(prefix="block1_")
pd.get("exciting_parameter_yay", shape=(3,3))
pd

block1_ (
  Parameter block1_exciting_parameter_yay (shape=(3, 3), dtype=<class 'numpy.float32'>)
)

In [86]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super().__init__(**kwargs)
        with self.name_scope():
            self.weight = self.params.get(
                'weight', shape=(in_units, units))
            self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [87]:
dense = MyDense(5, in_units=10, prefix='o_my_dense_')
dense.params

o_my_dense_ (
  Parameter o_my_dense_weight (shape=(10, 5), dtype=<class 'numpy.float32'>)
  Parameter o_my_dense_bias (shape=(5,), dtype=<class 'numpy.float32'>)
)

In [90]:
!nvidia-smi

Thu Apr 12 18:30:21 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 391.01                 Driver Version: 391.01                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 950M   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8    N/A /  N/A |    937MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  